# Preparation

## Imports

In [2]:
import os
import random
import json
import dotenv
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from pymongo.errors import CollectionInvalid, DuplicateKeyError
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from faker import Faker

## Connect to MongoDB

In [3]:
dotenv.load_dotenv()

mongo_user = os.getenv("MONGODB_USERNAME")
mongo_pass = os.getenv("MONGODB_PASSWORD")

mongo_client = MongoClient(
    (f"mongodb+srv://{os.getenv("MONGODB_USERNAME")}:{os.getenv("MONGODB_PASSWORD")}"
     "@projeto-bd.9scqvyv.mongodb.net/"
     "?retryWrites=true&w=majority&appName=projeto-bd"),
    server_api = ServerApi(
        version = "1",
        strict = True,
        deprecation_errors = True
    )
)

mongo_db = mongo_client["music_catalog"]

## Connect to Neo4j

In [4]:
dotenv.load_dotenv()

neo4j = GraphDatabase.driver(
    "neo4j+s://10ab7e50.databases.neo4j.io",
    auth = (
        os.getenv("NEO4J_USERNAME"),
        os.getenv("NEO4J_PASSWORD"),
    ),
)

neo4j.verify_connectivity()

## Connect to Spotify API

In [5]:
dotenv.load_dotenv()

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

## Init Faker

In [6]:
fake = Faker()

# Artists

## Create Entities

### MongoDB

In [7]:
try:
    mongo_db.create_collection("artists")
    mongo_db.artists.create_index("releases.id")
except CollectionInvalid as e:
    print(e)

### Neo4j

In [8]:
try:
    neo4j.execute_query("CREATE CONSTRAINT FOR (a:Artist) REQUIRE a.id IS UNIQUE")
    neo4j.execute_query("CREATE CONSTRAINT FOR (g:Genre) REQUIRE g.name IS UNIQUE")
    neo4j.execute_query("CREATE CONSTRAINT FOR (r:Release) REQUIRE r.id IS UNIQUE")
except ClientError as e:
    print(e.message)

An equivalent constraint already exists, 'Constraint( id=3, name='constraint_49e61036', type='NODE PROPERTY UNIQUENESS', schema=(:Artist {id}), ownedIndex=2 )'.


## Retrieve Artists by Genre

### From Spotify

In [ ]:
response = spotify.search(
    q = "genre:djent",
    type = "artist",
)
artist_ids = [artist["id"] for artist in response["artists"]["items"]]

while response["artists"]["next"]:
    response = spotify.next(response["artists"])
    artist_ids.extend([artist["id"] for artist in response["artists"]["items"]])

artist_ids

### From JSON

In [9]:
with open("../resources/my_guys.json", "r") as f:
    artist_ids = json.load(f)

artist_ids

['3Ri4H12KFyu98LMjSoij5V',
 '6vwjIs0tbIiseJMR3pqwiL',
 '6qqNVTkY8uBg9cP3Jd7DAH',
 '4zllGt4ePrRAIaWiEu5pyz',
 '1Ffb6ejR6Fe5IamqA5oRUF',
 '4yRSUmhuSJ3KcIMljdh4fH',
 '3P4vW5tzQvmuoNaFQqzy9q',
 '3l0CmX0FuQjFxr8SK7Vqag',
 '7kWnE981vITXDnAD2cZmCV',
 '6V267iCF72e4eP1L3ZkKQt',
 '4XpPveeg7RuYS3CgLo75t9',
 '0NbQe5CNgh4YApOCDuHSjb',
 '6qpmcQfxhmNxNMCJUrpHwe',
 '0GDGKpJFhVpcjIGF8N6Ewt',
 '7yYa2im7sawSzuVkXx8W21',
 '6sHCvZe1PHrOAuYlwTLNH4',
 '6LE9lW3E48cGM8tk5UGv30',
 '6FfjnGXMhxSsJTuGLWBDth',
 '6fb3I3Q54izgnOMtiZbOBA',
 '7nKz8GVqHk0bUGmBm6wm3E',
 '7rqJQQxuUOCk052MK5kLsH',
 '3ALVPmg5sZexSVD2m9atEt',
 '7o6cOczXTB8ioTAAJTbESf',
 '1ejkQAcOu9cl7kEbZ3Nb8b',
 '3MZsBdqDrRTJihTHQrO6Dq',
 '6XyY86QOPPrYVGvF9ch6wz',
 '4G9wSdX0klmoHfjm9i6DLd',
 '4qY6XGFQwZubu0oKBJeVki',
 '0LCI5aIo6Wd80D1AzHEu0F',
 '0P9pI1DLcVTkobNcZ7Tb1N',
 '27e4QBDvN4daYHHokUpWZY',
 '6Ai0kQ1MZABOQLVZNGozBB',
 '6MwPCCR936cYfM1dLsGVnl',
 '5BIBb9b6B9bKsebZFtgIVB',
 '77SW9BnxLY8rJ0RciFqkHh',
 '2kxP07DLgs4xlWz8YHlvfh',
 '6jqZZA8CPEkhFaVjBJ4Ctl',
 

## Insert into DBs

In [10]:
def release_tracks(release_id: str) -> list:
    response = spotify.album_tracks(release_id)

    tracks = []
    for track in response["items"]:
        tracks.append({
            "track_number": track["track_number"],
            "name": track["name"],
            "duration": track["duration_ms"]
        })

    while response["next"]:
        response = spotify.next(response)
        for track in response["items"]:
            tracks.append({
                "track_number": track["track_number"],
                "name": track["name"],
                "duration": track["duration_ms"]
            })

    return tracks

def artist_releases(artist_id: str) -> list:
    response = spotify.artist_albums(artist_id, album_type="album")

    releases = []
    for release in response["items"]:
        releases.append({
            "id": release["id"],
            "name": release["name"],
            "release_date": release["release_date"],
            "tracks": release_tracks(release["id"]),
            "ratings": [],
        })

    while response["next"]:
        response = spotify.next(response)
        for release in response["items"]:
            releases.append({
                "id": release["id"],
                "name": release["name"],
                "release_date": release["release_date"],
                "tracks": release_tracks(release["id"]),
                "ratings": [],
            })

    return releases

for artist_id in artist_ids:
    response = spotify.artist(artist_id)

    artist = dict()
    artist["_id"] = artist_id
    artist["name"] = response["name"]
    artist["genres"] = response["genres"]
    artist_popularity = response["popularity"]
    if random.random() < 0.9:
        artist["bio"] = fake.paragraph(nb_sentences=25)
    artist["qt_followers"] = 0
    artist["releases"] = artist_releases(artist_id)

    if len(artist["releases"]) > 0:
        mongo_db.artists.insert_one(artist)

        neo4j.execute_query(
            """
            MERGE (a:Artist {id: $id})
            ON CREATE SET a.popularity = $popularity
            """,
            id = artist["_id"],
            popularity = artist_popularity,
        )

        for genre in artist["genres"]:
            neo4j.execute_query(
                """
                MATCH (a:Artist {id: $artistId})
                MERGE (g:Genre {name: $name})
                MERGE (a)-[:BELONGS_TO]->(g)
                """,
                artistId = artist["_id"],
                name = genre,
            )

        for release in artist["releases"]:
            neo4j.execute_query(
                """
                MATCH (a:Artist {id: $artistId})
                MERGE (r:Release {id: $id})
                MERGE (a)-[:RELEASED]->(r)
                """,
                artistId = artist["_id"],
                id = release["id"],
            )

# Users

## Create Entities

### MongoDB

In [ ]:
try:
    mongo_db.create_collection("users")
    mongo_db.users.create_index("username", unique=True)
except CollectionInvalid as e:
    print(e)

### Neo4j

In [ ]:
try:
    neo4j.execute_query("CREATE CONSTRAINT FOR (u:User) REQUIRE u.username IS UNIQUE")
except ClientError as e:
    print(e.message)

## Insert into DBs

### Randoms

In [ ]:
QT_TRIES = 1_000
qt_fails = 0

for _ in range(QT_TRIES):
    try:
        first_name = fake.first_name()
        last_name = fake.last_name()
        number = random.randint(0, 99)

        user = dict()
        user["username"] = f"{first_name.lower()}_{last_name.lower()}{number:02}"
        user["password"] = fake.sha256()
        if random.random() < 0.75:
            user["name"] = f"{first_name} {last_name}"
        if "name" in user and random.random() < 0.5:
            user["bio"] = fake.paragraph(nb_sentences=10)
        user["friends"] = []
        user["artists_followed"] = []
        user["ratings"] = []

        mongo_db.users.insert_one(user)

        neo4j.execute_query(
            """
            MERGE (u:User {username: $username})
            """,
            username = user["username"],
        )
    except DuplicateKeyError:
        qt_fails += 1

print(f"Finished with {QT_TRIES - qt_fails} users inserted.")        

### Personalized

In [ ]:
user = dict()
user["username"] = f"sahudy"
user["password"] = fake.sha256()
user["name"] = f"Sahudy Montenegro González"
user["bio"] = (
"""Criançada,

me desculpem desde já, vou ter que cancelar a aula de amanhã. 

Sei que serão muito infelizes com essa notícia ;)

A aula prevista para amanhã é sobre o Redis, um in-memory KV store, bem simples de se iniciar. A aula está liberada no AVA, mas eu vou repor ela no último dia de aula, como previsto no calendário, já que não interfere na elaboração dos projetos.

Boa semana!

G5: apresentação na semana que vem :) acredito que esta decisão minha possa nterferir no andamento do projeto de vocês, me escrevam por email sobre suas preocupações, se houver alguma. 

Abs,"""
)
user["friends"] = []
user["artists_followed"] = []
user["ratings"] = []

mongo_db.users.insert_one(user)

neo4j.execute_query(
    """
    MERGE (u:User {username: $username})
    """,
    username = user["username"],
)

In [ ]:
user = dict()
user["username"] = f"ryansakurai"
user["password"] = fake.sha256()
user["name"] = f"Ryan Sakurai"
user["bio"] = "Sou Ryan, pessoa não binária, mestiço de raízes misturadas e pulsantes que carrego com orgulho no sangue, medindo 1,64m de pura teimosia e inquietude, apaixonado por metal, esse gênero musical que traduz em riffs e guturais toda a intensidade que sinto dentro do peito. Fã incondicional de One Piece, me identifico profundamente com as jornadas de superação, liberdade e amizade que a obra ensina, e confesso que choro mais do que gostaria em alguns arcos épicos — sim, meu lado nerd não falha em se emocionar com boas histórias. Entre um headbang e outro, vivo imerso em universos de fantasia, tecnologia e cultura pop, buscando sempre me expressar com autenticidade e sem medo de abraçar o que sou, por mais contraditório ou peculiar que isso possa parecer aos olhos dos outros."
user["friends"] = []
user["artists_followed"] = []
user["ratings"] = []

mongo_db.users.insert_one(user)

neo4j.execute_query(
    """
    MERGE (u:User {username: $username})
    """,
    username = user["username"],
)

In [ ]:
user = dict()
user["username"] = f"viniciuscastro"
user["password"] = fake.sha256()
user["name"] = f"Vini"
user["bio"] = "Sou o Vini, um cara que domina o mundo das construções, mecânica e eletricidade, combinando força de pedreiro, precisão de mecânico e a paciência de eletricista — mesmo com minhas pernas finas, nada me segura quando o serviço chama. Sou funkeiro de alma, aquele que sente o batidão no corpo e na mente, mas não perco tempo com Instagram porque minha vida é muito real pra ficar postando tudo por aí. Faculdade? Ah, essa fica meio de lado, porque prefiro mil vezes acelerar na direção da minha rotina, dirigindo igual maluco pelas ruas e ainda surpreendendo com umas manobras de Python que aprendi na marra, misturando o mundo do código com a vida de obra e estrada. Minha vibe é fazer acontecer, mesmo que no ritmo acelerado e fora do convencional, porque ser do jeito que sou é o que me mantém firme e único."
user["friends"] = []
user["artists_followed"] = []
user["ratings"] = []

mongo_db.users.insert_one(user)

neo4j.execute_query(
    """
    MERGE (u:User {username: $username})
    """,
    username = user["username"],
)

In [ ]:
user = dict()
user["username"] = f"caike_sant0s"
user["password"] = fake.sha256()
user["name"] = f"Caique (Hugo)"
user["bio"] = "Eu sou Caike, mas pode me chamar de Hugo ou Cacá, apelidos que me acompanham nas minhas aventuras de rolezeiro nato, sempre buscando o próximo encontro, a próxima história pra contar. Tenho uma irmã que, apesar das brigas e zoações, é parceira de vida e cúmplice das melhores confusões. Andar de moto é mais que um hobby, é um estilo de vida, a sensação de liberdade que só quem já sentiu o vento no rosto entende. Sempre sigo meu lema: existem dois tipos de problemas — o meu e o dos outros — e prefiro focar no que posso resolver, sem me perder nas dores alheias. Ah, e não é pouca coisa não, sou o aluno preferido da professora Sahudy, aquela que sabe reconhecer quando alguém leva as coisas a sério, mesmo que eu esteja sempre no meio do rolê."
user["friends"] = []
user["artists_followed"] = []
user["ratings"] = []

mongo_db.users.insert_one(user)

neo4j.execute_query(
    """
    MERGE (u:User {username: $username})
    """,
    username = user["username"],
)

# Close Connections

In [ ]:
mongo_client.close()
neo4j.close()